In [73]:
import os
import glob

def list_file_paths(directory):
    file_paths = [f for f in glob.glob(directory + "/**/*", recursive=True) if os.path.isfile(f)]
    return file_paths

folder = 'walking_side' #change here

directory_path = "E:\\Download\\uav_actions\\Cropped\\"+folder+"\\"
file_paths = list_file_paths(directory_path)
list_vid = []

In [74]:
for path in file_paths:
    if 'mp4' in path:
        tmp = path.split('\\')
        # print(tmp)
        list_vid.append("\\"+tmp[5])

In [ ]:
list_vid

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize video capture and output.
input_video_path = directory_path + list_vid[0]
print(input_video_path)
output_video_path = 'E:\\Download\\uav_actions\\Cropped\\'+folder+".mp4"
cap = cv2.VideoCapture(input_video_path)

# Get video properties.
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object.
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Function to calculate the angle between three points.
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.abs(np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0]))

    if radians > np.pi:
        radians = 2*np.pi - radians

    return radians

In [77]:
from tqdm import tqdm

def get_angle_seq(list_vid, path, tar):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    total_step = len(list_vid)
    pbar = tqdm(total=total_step)

    for path in list_vid:
        # os.makedirs(tar + path.split('.')[0], exist_ok=True)
        
        source = directory_path + path
        print(source)
        target = tar + path.split('.')[0] + '.txt'
        cap = cv2.VideoCapture(source)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        angles_list = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            try:
                results = pose.process(image_rgb)
            except:
                print(path)
                continue

            frame_angles = []
            cv2.imshow('frame', frame)
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # List of points for angle calculation.
                points = [
                    # Upper body
                    (11, 13, 15),  # Left Shoulder-Elbow-Wrist
                    (12, 14, 16),  # Right Shoulder-Elbow-Wrist
                    (12, 11, 13),
                    (11, 12, 14),
                    (13, 11, 23),
                    (14, 12, 24),
                    (24, 23, 25),
                    (23, 24, 26),
                    # Torso and lower body
                    (23, 25, 27),  # Left Hip-Knee-Ankle
                    (24, 26, 28),  # Right Hip-Knee-Ankle
                    (11, 23, 25),  # Left Shoulder-Hip-Knee
                    (12, 24, 26),  # Right Shoulder-Hip-Knee
                    (23, 25, 27),  # Left Hip-Knee-Foot Index
                    (24, 26, 28),  # Right Hip-Knee-Foot Index
                ]

                for (a_idx, b_idx, c_idx) in points:
                    a = [landmarks[a_idx].x * frame_width, landmarks[a_idx].y * frame_height]
                    b = [landmarks[b_idx].x * frame_width, landmarks[b_idx].y * frame_height]
                    c = [landmarks[c_idx].x * frame_width, landmarks[c_idx].y * frame_height]

                    cv2.line(frame, (int(a[0]), int(a[1])), (int(b[0]), int(b[1])), (0, 255, 0), 2)
                    cv2.line(frame, (int(b[0]), int(b[1])), (int(c[0]), int(c[1])), (0, 255, 0), 2)

                    angle = calculate_angle(a, b, c)

                    # Draw landmarks and angles on the frame.
                    # cv2.circle(frame, (int(b[0]), int(b[1])), 5, (0, 255, 0), -1)
                    # cv2.putText(frame, f'{int(angle)}', (int(b[0]), int(b[1] - 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
                    frame_angles.append(angle)
                    # print(angle)

            # Write the frame with the angle.
            angles_list.append(frame_angles)

            # cv2_imshow(frame)

        # Release resources.
        cap.release()
        # pose.close()
        cv2.destroyAllWindows()


        with open(target, 'w') as file:
            for frame_angles in angles_list:
                file.write(','.join(map(str, frame_angles)) + '\n')
        pbar.update(1)
        # break
    pbar.close()

In [ ]:
list_vid

In [ ]:
get_angle_seq(list_vid, 'E:\\Download\\uav_actions\\Cropped\\'+folder+'\\', 'E:\\Download\\uav_actions\\Extracted\\'+folder+'\\')